In [62]:

from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import os
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import lxml

In [2]:
index_url='https://sipmen.bps.go.id/regsosek/cetak-box/index-generate-box-kab'
first_page=index_url+'?page=1&search='
login_url='https://sipmen.bps.go.id/regsosek/login'

# user login credential
email='johndoea@gmail.com'
password="password"

In [ ]:
driver = webdriver.Chrome('./chromedriver')
driver.get(first_page)

time.sleep(1)

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# simulasi login
if(driver.current_url==login_url):
    wait = WebDriverWait(driver, 1)
    wait.until(EC.element_to_be_clickable((By.ID, "email"))).send_keys(email)
    wait.until(EC.element_to_be_clickable((By.ID, "password"))).send_keys(password)
    wait.until(EC.element_to_be_clickable((By.XPATH, "//button[text()=' Sign In']"))).click()
    
time.sleep(2)

no_batch = []
kode_sls = []
urutan_batch = 1
urutan_sls=1
# halaman index/ halaman pertama
if(driver.current_url==first_page):
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    soup_table = soup.find("table")
    
    table = pd.read_html(str(soup_table), converters ={x:str for x in ['NOMOR BATCH', 'RINCIAN DOKUMEN']})[0]
    # ambil no batch
    no_batches = table.iloc[:,0]

    # untuk setiap nomor batch (kita perlu indexnya)
    for i in range(len(no_batches)):  
    # ambil rincian dokumen, ekstrak ul dulu, trus ektrak li
        ul = soup_table.find_all('ul')[i]
        lis = ul.find_all('li')
        x = 1

        for li in lis:
            no_batch.append(no_batches[i])
            kode_sls.append(li.text.strip())
            print('halaman 1, no batch: ',i,'kode sls',x,'/',len(lis))
            print('overall :', 'halaman 1, no batch ke-',urutan_batch,',kode sls ke-',urutan_sls)
            x=x+1

    urutan_sls=urutan_sls+1
    urutan_batch = urutan_batch+1


    #halaman kedua dst
    dataIsAvailable = True
    page=2
    while(dataIsAvailable):

        url = index_url+'?page='+str(page)+'&search='

        driver.get(url)
        time.sleep(3)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        soup_table = soup.find("table")
        table = pd.read_html(str(soup_table), converters ={x:str for x in ['NOMOR BATCH', 'RINCIAN DOKUMEN']})[0]
        # kalau halaman terakhir, maka cuma ada satu row, nilainya 'tidak ditemukan'

        if(len(table.index)<=1):
            dataIsAvailable=False
            columns = ['no batch', 'kode sls']
            data = np.array([no_batch, kode_sls])
            table_final = pd.DataFrame(data=data, columns=columns)
            table_final.to_csv('sipmen_batching')
        else:
            # 
            no_batches = table.iloc[:,0]
            # untuk setiap nomor batch (kita perlu indexnya)
            for i in range(len(no_batches)):  
            # ambil rincian dokumen, ekstrak ul dulu, trus ektrak li
                ul = soup_table.find_all('ul')[i]
                lis = ul.find_all('li')
                x=1
                for li in lis:
                    no_batch.append(no_batches[i])
                    kode_sls.append(li.text.strip())
                    print('halaman 1, no batch: ',i,'kode sls',x,'/',len(lis))
                    print('overall :', 'halaman 1, no batch ke-',urutan_batch,',kode sls ke-',urutan_sls)
                    x=x+1
            urutan_sls=urutan_sls+1
            urutan_batch = urutan_batch+1
        page=page+1
        
    driver.close()
